In [1]:
import numpy as np
import xarray as xr
import geocat.comp as gc

# The Problem

While using GeoCAT's `linint2` to interpolate between CAM grids and satellite product grids (both regular lat-lon), I find that `linint2` usually does not behave as I expect. Instead of interpolating across the prime meridian, I keep finding a `nan` at all longitudes. 

Example:

In [3]:
# Make fake data array of all ones on a regular lat-lon grid
fakedata = xr.DataArray(np.ones((90,180)), dims=("lat","lon"), coords={"lat":np.linspace(-89, 89, 90), "lon": np.linspace(0, 360-2, 180)})
print(fakedata.coords)

# interpolate to 1 degree grid

# try without specifying that it is cyclic:
fakeone = gc.linint2(fakedata, np.linspace(0, 360-1, 360), np.linspace(-89.5, 89.5, 180), fakedata.lon, fakedata.lat, icycx=False).compute()

# try with the flag that says longitude is cyclic:
fakeone_cyc = gc.linint2(fakedata, np.linspace(0, 360-1, 360), np.linspace(-89.5, 89.5, 180), fakedata.lon, fakedata.lat, icycx=True).compute()

# interpolate to a 4 degree grid
fakefour = gc.linint2(fakedata, np.linspace(0, 360-4, 360//4), np.linspace(-88, 88, 45), fakedata.lon, fakedata.lat, icycx=False).compute()
fakefour_cyc = gc.linint2(fakedata, np.linspace(0, 360-4, 360//4), np.linspace(-88, 88, 45), fakedata.lon, fakedata.lat, icycx=True).compute()


Coordinates:
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0


TypeError: 'NoneType' object is not iterable

In [ ]:

print(fakeone.values)  # 1 degree, icycx=False  ==> There are nans around the edges of the interpolation

print(fakeone_cyc.values)  # 1 degree, icycx=True ==> this one has weird values!

print(fakefour.values)


In [ ]:
def linint_wrap(fi, xo, yo):
    addpt = fi.isel(lon=0).copy()
    addpt['lon'] = fi.lon[-1]+(fi.lon[1]-fi.lon[0])
    xiwrap = xr.concat([fi, addpt], dim='lon')
    return gc.linint2(xiwrap, xo, yo, xi=xiwrap.lon, yi=xiwrap.lat, icycx=1, msg_py=None).compute()

In [ ]:

#
# use linint_wrap
#
fakeone_wrap = linint_wrap( fakedata, np.linspace(0, 360-1, 360), np.linspace(-89.5, 89.5, 180)) 